<a href="https://colab.research.google.com/github/gweber/collab_experiments/blob/master/flappy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#flappy.py

import pygame as pg
import numpy as np
import colorsys
import math
import random
from copy import deepcopy
random_seed = 1

import torch
import torch.nn as nn
import torch.nn.functional as F


class Neural_Network(nn.Module):

    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 4
        self.hiddenSize = 6 # change to 1 to have in -> out, output of forward chane to z2 then
        self.outputSize = 1
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 
        self.z2 = self.sigmoid(self.z) # activation function

        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)

    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))



class Bird:
	def __init__(self,color,x_offset = 0, born_by = 'random', id=0):
    # bird is dropped in the middle vertically, and on its position horizontaly
		self.pos = [ 200 + x_offset, settings['screen_size']['y']/2 ]
		self.color = color
		# which direction, 1 follow gravity, -1 jump
		self.direction = 1
		self.radius = 10 #int(random.uniform(10,30))
		#self.layer_sizes = (4, 6, 1)
		self.flaps_left = 0
		self.born_in = loop
		self.born_by = born_by
		self.net = Neural_Network()
		self.die = 0
		self.id = id

	def check_health(self):
		if loop - self.born_in < 10:
			return 0
		if self.pos[1] > settings['screen_size']['y']:
			self.status = -1
			return -1
		if self.pos[1] < 0:
			self.status = -1
			return -1
		# collision?
		# create a rectanle for the circle ...
		coll = pg.Rect(self.pos[0] - self.radius, self.pos[1] - self.radius, 2 * self.radius, 2 * self.radius)
		for obstacles in rect_obstacles:
			# check top [0] and buttom[2]
			if obstacles[0].colliderect(coll) or obstacles[2].colliderect(coll):
				self.status = -1
				return -1

	def update_pos(self, amount = 10, direction = 0):
		if not direction:
			direction = self.direction
		if self.flaps_left > 0:
			self.flaps_left -= 1
			direction = -1
			amount =  ( self.flaps_left * self.flaps_left ) * 10 / self.radius * 0.25 
			#print('update_pos: flapping')
		self.pos[1] += direction * amount 

	def flap(self):
		if self.flaps_left == 0:
			self.flaps_left = 10
			#print('flap: flapping')
		else:
			#print('flap: cant flap')
			pass
		self.update_pos()

	def check_distance(self, obstacles):
		next_obstacle = 0
		for i,obstacle in enumerate(rect_obstacles):
			# have i passed the obstacle already, check vs top [0]
			if self.pos[0] > obstacles[next_obstacle][0].right + self.radius:
				next_obstacle = i + 1
    # get the distance to the tunnel
		(x_pos, y_pos) = rect_obstacles[next_obstacle][1].center
		x_distance = x_pos - self.pos[0]
		y_distance = y_pos - self.pos[1]
		tunnel_height = rect_obstacles[next_obstacle][1].height
		# y_distance > 0: hole is below bird
		# self.pos: distance from top or buttom 
		predict = self.net.forward(
			torch.tensor([x_distance, y_distance, tunnel_height, self.pos[1]])
		)
		
		if (predict) >= 0.5:

			#print('check_distance: wanna flap')
			self.flap()
		#print(x_distance, y_distance)		 


def draw_net(bird):
	top_left = 600,0
	for bias in bird.net.biases[0]:
		pass

def move_rect_obstacles(rect_obstacles, x_offset):
	for idx, obstacle in enumerate(rect_obstacles):
		for j,__ in enumerate(obstacle):
			rect_obstacles[idx][j] = obstacle[j].move( -x_offset,0)
	return rect_obstacles

def draw_rect_obstacles(rect_obstacles):
	color = (0,127,255)
	for obstacles in rect_obstacles:
		pg.draw.rect(screen, color, (obstacles[0]))
		pg.draw.rect(screen, color, (obstacles[2]))

def draw_circle(pos=(10,10), color=(255,0,0), radius=20):
	pos = [int(i) for i in pos]
	pg.draw.circle(screen, color, pos, radius )

def generate_obstacle(height, variance=0, blank=50):
	obstacle = {}
	return random.randint(0 + blank, height - blank - settings['obstacle_tunnel'] - variance)


def draw_bird(bird):
	draw_circle(pos = bird.pos, color = bird.color, radius = bird.radius)

def draw_birds(birds):
	for bird in birds:
		draw_bird(bird)

def check_distance():
	pass


def main():
	global running, screen, bird, settings, loop, obstacles, rect_obstacles, archive
	loop = 0
	
	up = -1
	down = 1
	
	rect_obstacles = []
	birds = []
	archive = {}
	# id: age at birth / born_in
	# fitness: age it reached
	# weights: the weights

	max_fitness = 0

	dead_birds = 0


	next_algo = ['mutate', 'mutate','mutate','random','random','random','random', 'crossover','crossover''crossover''crossover''crossover''crossover']

	settings = {
		'screen_size': {'x': 1800, 'y': 600},
		'obstacle_distance': 200,
		'speed_factor': 1,
		'obstacle_width': 30,
		'obstacle_tunnel': 150,
		'tunnel_variance': 30,
		'speedup': 0,
		'tunnel_decrease': 0,
		'bird_radius': 10,
		'sleep': 10, # sleep ms
		'birds': 20, # count of birds
		'speed_per_tick': 5,
		'spacer': 20,
		'gravity': 10
	}


	pg.init()

	for i in range(0,settings['birds']):
		birds.append(Bird(np.random.randint(0,255,3), x_offset = i*45, born_by='random', id=dead_birds ))

	screen = pg.display.set_mode((settings['screen_size']['x'], settings['screen_size']['y']))
	pg.display.set_caption("nnflap")
	
	running = True
	while running:

		
		loop -= -1
		screen.fill((0,0,0))
		for event in pg.event.get():
			if event.type == pg.QUIT:
				running = False
			if event.type == pg.KEYDOWN:
				if event.key == pg.K_LEFT:
					x_pos -= 10
				if event.key == pg.K_RIGHT:
					x_pos += 10
				if event.key == pg.K_UP:
					if settings['sleep'] > 0: 
						settings['sleep'] -= 1
				if event.key == pg.K_DOWN:
					settings['sleep'] += 1
				if event.key == pg.K_SPACE:
					running = False
		

		# check if first obstacle is obsolete
		if len(rect_obstacles) and rect_obstacles[0][0].right < 0:
			# if obstacle is obsolete, create new copy of obstacles
			rect_obstacles = rect_obstacles[1:]


		# check distance to last obstacle
		# if last obstacle is into the screen and more than distance into it create a new obstacle
		if len(rect_obstacles) == 0 or rect_obstacles[-1][0].right < settings['screen_size']['x'] - settings['obstacle_distance']:

			variance = int( random.uniform(-1,1) * settings['tunnel_variance'] ) 
			tunnel_from_top = generate_obstacle(settings['screen_size']['y'], blank=settings['spacer'])

			rect_top = pg.Rect(settings['screen_size']['x'], 0, settings['obstacle_width'], tunnel_from_top )
			
			rect_tunnel = pg.Rect(settings['screen_size']['x'], tunnel_from_top, settings['obstacle_width'], settings['obstacle_tunnel'] + variance)

			rect_buttom = pg.Rect(settings['screen_size']['x'], tunnel_from_top + settings['obstacle_tunnel'] + variance, settings['obstacle_width'], settings['screen_size']['y'] )
			
			rect_obstacles.append([rect_top, rect_tunnel, rect_buttom])
		
		rect_obstacles = move_rect_obstacles(rect_obstacles, settings['speed_per_tick'])




		best_birds = []
		for bird_index, bird in enumerate(birds):
			# let the bird sink
			bird.update_pos(amount = settings['gravity'], direction = down)
			# bird fitness is ... age?
			best_birds.append([bird_index, loop - bird.born_in])


		best_fitness = sorted(best_birds, key=lambda x: x[1], reverse=True)


		for check_archive in best_fitness:
			#fetch the 10 best from archive
			#copare if fitness is better

			# check if same bird already in list
			if 'fit'+str(birds[check_archive[0]].id) in archive:
				# update the old entry
				archive['fit'+str(birds[check_archive[0]].id)]['fitness'] = check_archive[1]


			if check_archive[1] > max_fitness:
				max_fitness = check_archive[1]
				archive['fit'+str(birds[check_archive[0]].id)] = {'id': birds[check_archive[0]].id }
				archive['fit'+str(birds[check_archive[0]].id)]['fitness'] =  check_archive[1]
				archive['fit'+str(birds[check_archive[0]].id)]['net'] =  deepcopy(birds[check_archive[0]].net)
				print('archive', archive['fit'+str(birds[check_archive[0]].id)])
		#		print('max fit:', max_fitness)

		
		
		#exit()


		
		stats = {}
		for bird_index, bird in enumerate(birds):
			# bird has died
			if bird.check_health() == -1 or bird.die == 1:
				dead_birds -= -1
				birds[bird_index] = Bird(np.random.randint(0,255,3), x_offset = bird_index * 45, id=dead_birds)
				if (next_algo[0] == 'random'):
					birds[bird_index].born_by = 'random'
					#print('randomized')
					pass

				if (next_algo[0] == 'mutate'):
					#print('mutated\n')
					# get the best net so far
					max_test = { j['fitness']: i for i, j in archive.items() }
					#archive[max_test[max(max_test.keys())]].net
					bird.net = deepcopy(archive[max_test[max(max_test.keys())]]['net'])
					birds[bird_index].born_by = 'mutate'
					
					bird.net.W1 += torch.randn(bird.net.W1.shape)/1000
					bird.net.W2 += torch.randn(bird.net.W2.shape)/1000
					"""
					print(
						'ar w1: ', archive[max_test[max(max_test.keys())]]['net'].W1, "\n",
						'ar w2: ', archive[max_test[max(max_test.keys())]]['net'].W2, "\n",
						'b w1: ', bird.net.W1, "\n",
						'b w2: ', bird.net.W2, "\n",
					)
					"""
			
				"""

			

				if (next_algo[0] == 'crossover'):
					birds[bird_index].born_by = 'crossover'
					# get a sample of best birds
					crossover = random.sample(best_fitness[5:], 2)
					for i1,w in enumerate(bird.net.weights):
						for i2,x in enumerate(w):
							for i3,y in enumerate(x):
								if random.uniform(0,1) <= 0.5: # take from the best
									bird.net.weights[i1][i2][i3] = copy.copy(birds[crossover[0][0]].net.weights[i1][i2][i3])
								else:
									bird.net.weights[i1][i2][i3] = copy.copy(birds[crossover[1][0]].net.weights[i1][i2][i3])
					for i1, b in enumerate(bird.net.biases):
						for i2, x in enumerate(b):
							if random.uniform(0,1) <= 0.5: # take from the best
								#bird.net.biases[i1][i2] = birds[crossover[0][0]].net.biases[i1][i2]
								pass
							else: 
								#bird.net.biases[i1][i2] = birds[crossover[1][0]].net.biases[i1][i2]
								pass
"""
			random.shuffle(next_algo)

			bird.check_distance(rect_obstacles)

			pg.draw.line(screen, bird.color, 
				( 0, settings['screen_size']['y'] - 1 - bird_index ), 
				( loop-bird.born_in, settings['screen_size']['y'] - 1 - bird_index ),
			)


		for i,best in enumerate(best_fitness[:4]):
			pg.draw.polygon(
				screen, 
				(255-i*32, 255-i*32, 255-i*32), 
				[ 
					[ 200 + best[0] * 45 - 30 , settings['screen_size']['y']],
					[ 200 + best[0] * 45      , settings['screen_size']['y'] - 52],
					[ 200 + best[0] * 45 + 30 , settings['screen_size']['y']]
				]
			)

		# pg stuff
		
		draw_rect_obstacles(rect_obstacles)
		draw_birds(birds)
		
		

		pg.display.update()
		pg.time.delay(settings['sleep'])
		# add 5% speed each 1k it
		#print(loop, settings['speed_factor'])
		
		#print(birds[best_fitness[0][0]].net.W1)
		if (loop % 1_000) == 0:
			print('loop:', loop, 'dead birds:', dead_birds)
			settings['speed_factor'] += settings['speed_factor']
			settings['obstacle_tunnel'] -= settings['tunnel_decrease']

if __name__ == '__main__':
    main()

     |████████████████████████████████| 11.4MB 203kB/s 
pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


error: ignored